260107.17:31
This file create for complete new globalaggregation class that can call gemini

In [1]:
import re
import yaml
import json
import os

class LiteralString(str):
    """Force YAML to use block literal style '|'."""
    pass
def literal_representer(dumper,value):
    """Represent LiteralString using YAML block style."""
    return dumper.represent_scalar('tag:yaml.org,2002:str', value, style='|')

# Register custom representer
yaml.add_representer(LiteralString, literal_representer)

class Helper:
    """
    Utility helper class for file I/O, YAML/JSON handling,
    formatting, and configuration transformations.
    """
    @staticmethod
    def load_file(filepath: str) -> str:
        """
        Load and return the contents of a text file.
        Args:
            filepath (str): Path to the file.
        Returns:
            str: File contents.
        """
        with open(filepath, "r", encoding="utf-8") as file:
            return file.read()
        
    @staticmethod
    def load_yaml(filepath: str) -> dict:
        """
        Load a YAML file and return its contents as a dictionary.
        Args:
            filepath (str): Path to the YAML file.
        Returns:
            dict: Parsed YAML content.
        """
        with open(filepath, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)
        
    @staticmethod
    def load_json(filepath: str) -> dict:
        """
        Load a JSON file and return its contents as a dictionary.
        Args:
            filepath (str): Path to the JSON file.
        Returns:
            dict: Parsed JSON content.
        """
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
        
    @staticmethod
    def save_yaml(newconfig,filepath: str):
        """
        Save a dictionary to a YAML file.
        Preserves key order and applies custom representers
        such as block literal formatting.
        Args:
            newconfig (dict): Configuration data to save.
            filepath (str): Output YAML file path.
        """
        with open(filepath,"w",encoding="utf-8") as file:
            return yaml.dump(newconfig,file,sort_keys=False)
        
    @staticmethod
    def fop(num: float) -> float:
        """
        Format a number to one decimal place.
        Args:
            num (float): Input number.
        Returns:
            float: Rounded number with one decimal precision.
        """
        return float(f"{num:.1f}")
    
    @staticmethod
    def prettyjson(txt:str) -> str:
        """
        Pretty-print a JSON-compatible object as a formatted string.
        Args:
            txt (str): JSON-compatible object.
        Returns:
            str: Indented JSON string.
        """
        return str(json.dumps(txt,indent=4, ensure_ascii=False))
    
    @staticmethod
    def to_literal(value):
        """
        Convert multiline strings into LiteralString for YAML output.
        Single-line strings and non-string values are returned unchanged.
        """
        if isinstance(value,str) and "\n" in value:
            return LiteralString(value)
        return value
    
    @staticmethod
    def deep_literal_transform(data):
        '''
        Recursively convert ALL multiline strings inside dict/list
        into LiteralString so block formatting is preserved
        '''
        if isinstance(data, dict):
            return {k: Helper.deep_literal_transform(v) for k,v in data.items()}
        if isinstance(data, list):
            return [Helper.deep_literal_transform(i) for i in data]
        # Convert only multiline string
        return Helper.to_literal(data)

In [2]:
from google import genai
import json
import os
import yaml

class PromptBuilder(Helper):
    """
    Builds a structured evaluation prompt for a specific resume section.
    Uses YAML-driven configuration to assemble role instructions,
    evaluation criteria, scoring scale, expected content, and
    the candidate resume into a single LLM-ready prompt.
    """

    def __init__(self, section, criteria, targetrole, cvresume, include_fewshot: bool = True, output_lang = "en"):
        """
        Initialize the prompt builder for a resume section.
        Args:
            section (str): Resume section to evaluate (e.g. "Education").
            criteria (list): Evaluation criteria for the section.
            targetrole (str): Target role used for role relevance.
            cvresume (str): Resume content to be evaluated.
            include_fewshot (bool): Whether to include example scores.
            output_lang (str): Output language code (e.g. "en", "th").
        """
        self.section        = section
        self.criteria       = criteria[::-1]
        self.cvresume       = cvresume
        self.targetrole     = targetrole
        self.include_fewshot= include_fewshot
        self.output_lang    = output_lang
        
        self.config = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\prompt.yaml")
        self.criteria_cfg = self.config.get("criteria", {})

    def build_response_template(self):
        """
        Build an empty JSON response template for the LLM.
        Returns:
            dict: Response skeleton with section name and criteria scores.
        """
        return {
            "section": self.section,
            "scores": {
                c: {"score": 0, "feedback": ""} for c in self.criteria
            }
        }
    
    def _build_criteria_block(self) -> str:
        """
        Construct the criteria description block for the prompt.
        Includes few-shot score examples if enabled and available
        in the prompt configuration.
        Returns:
            str: Formatted criteria block text.
        """
        blocks = []
        for crit in self.criteria:
            block = f"- {crit}\n"
            if self.include_fewshot and crit in self.criteria_cfg:
                few_cfg = self.criteria_cfg[crit]
                for score in [5, 3, 1]:
                    key = f"score{score}"
                    if key in few_cfg:
                        text = few_cfg[key].strip()
                        block += f"    score {score}: {text}\n"
            blocks.append(block)
        return "".join(blocks)
    
    def build(self):
        """
        Assemble the full evaluation prompt.
        Combines role instructions, evaluation objectives, section context,
        criteria definitions, scoring scale, expected output format,
        and resume content into a single prompt string.
        Returns:
            str: Final prompt ready to be sent to the LLM.
        """
        config_role      = self.config['role']['role1']
        config_objective = self.config['objective']['objective1']
        config_section   = self.config['section']['section1']
        config_expected  = self.config['expected_content'][self.section]
        config_scale     = self.config['scale']['score1']
        criteria_block   = self._build_criteria_block()
        config_lang      = self.config['Language_output_style'][self.output_lang]

        prompt_role      = f"Role :\n{config_role}\n\n"
        prompt_objective = f"Objectvie :\n{config_objective}\n"
        promnt_lang      = f"Output Language Instruction::\n{config_lang}\n"
        prompt_section   = f"Section :\n{config_section}\n\n"
        prompt_expected  = f"Expected :\n{config_expected}\n"
        prompt_criteria  = f"Criteria :\n{criteria_block}\n"
        prompt_scale     = f"Scale :\n{config_scale}\n"
        prompt_output    = f"Otput :\n{json.dumps(self.build_response_template(), indent=2)}\n\n"
        prompt_cvresume  = f"CV/Resume: \n{self.cvresume}\n"
        prompt = (
            prompt_role + prompt_objective + prompt_section + promnt_lang
            + prompt_expected + prompt_criteria + prompt_scale
            + prompt_output + prompt_cvresume 
        )
        # print(f"prompt -> \n{prompt}")
        prompt = prompt.replace("<section_name>", self.section)
        prompt = prompt.replace("<targetrole>", self.targetrole)
        return prompt

In [ ]:
import asyncio

from pydantic import BaseModel, Field, ValidationError
from typing import Dict

class CriterionScore(BaseModel):
    """
    Represents a single evaluation score for one criterion.
    Attributes:
        score (int): Integer score between 0 and 5.
        feedback (str): Textual feedback explaining the score.
    """
    score: int = Field(ge=0, le=5)
    feedback: str

class SectionEvaluation(BaseModel):
    """
    Validated LLM evaluation output for a resume section.
    Attributes:
        section (str): Name of the resume section.
        scores (Dict[str, CriterionScore]): Mapping of criteria to scores.
    """
    section: str
    scores: Dict[str, CriterionScore]

class LlmCaller(Helper):
    def __init__(self):
        """
        Handles interaction with the LLM for section-level resume evaluation.
        Responsibilities:
        - Send prompts to the LLM
        - Parse and validate structured JSON responses
        - Retry with a repair prompt if validation fails
        - Return validated output along with raw LLM response
        """
        self.model_cfg = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\model.yaml")
        self.client    = genai.Client(api_key='AIzaSyARN...b-3dHAWb5urRk')
        self.model     = self.model_cfg["model"]["generation_model"]
        self.usd2bath  = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\global.yaml")['currency']['USD_to_THB']
        self.log_digit = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\global.yaml")['logging']['logging_round_digit']

    def _parse(self, resp):
        """
        Parse raw LLM response text into a JSON object.
        Removes markdown code fences and converts the result to a dictionary.
        Args:
            resp: Raw response object from the LLM client.
        Returns:
            dict: Parsed JSON output.
        """
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        return json.loads(text)
    
    def _call_raw(self, prompt: str):
        """
        Send a prompt to the LLM without validation.
        Args:
            prompt (str): Prompt text to send.
        Returns:
            tuple:
                - Parsed JSON output (dict)
                - Raw LLM response object
        """
        resp = self.client.models.generate_content(
            model=self.model,
            contents=prompt
        )
        parsed = self._parse(resp)
        return parsed, resp
    
    def _validate(self, raw_output:dict)->SectionEvaluation:
        """
        Validate LLM output against the expected schema.
        Args:
            raw_output (dict): Parsed LLM output.
        Returns:
            SectionEvaluation: Validated evaluation object.
        Raises:
            ValidationError: If the output does not match the schema.
        """
        return SectionEvaluation.model_validate(raw_output)
    
    def _repair_prompt(self, error_msg: str) -> str:
        """
        Generate a corrective prompt when LLM output validation fails.
        Instructs the LLM to strictly follow the expected JSON schema.
        Args:
            error_msg (str): Validation error message.
        Returns:
            str: Repair prompt to prepend to the original prompt.
        """
        return f"""
                Your previous response was INVALID.
                Validation error:
                {error_msg}
                STRICT RULES:
                - Return JSON only
                - No markdown
                - No explanation
                - Follow schema exactly
                - Section name must start with a capital letters (e.g. "Education")
                Expected format:
                {{
                    "section": "<section_name>",
                    "scores": {{
                        "<criterion>": {{ "score": 0-5, "feedback": "string" }}
                    }}
                }}
        """
    
    def call(self,prompt:str, max_retry:int = 3):
        """
        Call the LLM with validation and automatic retry.
        Attempts to validate the LLM response. If validation fails,
        retries using a repair prompt up to the specified limit.
        Args:
            prompt (str): Prompt text to send.
            max_retry (int): Maximum number of retry attempts.
        Returns:
            tuple:
                - Validated evaluation result (dict)
                - Raw LLM response object
        """
        last_error = None
        repair_prompt = "\n"
        for attemp in range(max_retry):
            final_prompt = repair_prompt + prompt
            # print(f"final_prompt attemp : {attemp} -> \n {final_prompt}")
            output, raw = self._call_raw(final_prompt)
            try:
                validated = self._validate(output)
                # print('Status : 1')
                return validated.model_dump(),raw
            except ValidationError as e:
                last_error = str(e)
                repair_prompt = self._repair_prompt(last_error)
                # print('Status : 0')
                print("Output error recall again ...")
            finally:
                print('='*100)
        return {
            "section":"UNKNOW",
            "scores":{}
        },raw

    async def call_async(self, prompt: str):
        """
        Asynchronous wrapper for `call`.
        Executes the blocking LLM call in a background thread.
        Args:
            prompt (str): Prompt text to send.
        Returns:
            tuple:
                - Validated evaluation result (dict)
                - Raw LLM response object
        """
        return await asyncio.to_thread(self.call, prompt)


In [6]:
from datetime import datetime,timezone,timedelta
import copy

class SectionScoreAggregator(Helper):
    """
    Aggregates raw LLM criterion scores for a single resume section
    into weighted scores and a section-level total score (0-5).
    Configuration:
        - src/config/weight.yaml
    Expected LLM Output Format (input):
        {
            "section": "<section_name>",
            "scores": {
                "<criterion>": {
                    "score": int,        # 0-5
                    "feedback": str
                }
            }
        }

    Output Format:
        {
            "section": "<section_name>",
            "total_score": float,
            "scores": {
                "<criterion>": {
                    "score": float,      # weighted score
                    "feedback": str
                }
            }
        }
    """
    def __init__(self):
        self.config          = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\weight.yaml")         # config
    def aggregate(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }
    
    

In [70]:
class newga(LlmCaller,Helper):
    def __init__(self,SectionScoreAggregator_output:list):
        super().__init__()
        self.section_outputs = SectionScoreAggregator_output
        self.timestamp       = str(datetime.now(tz=(timezone(timedelta(hours=7)))))
        self.model_config    = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\model.yaml")     # should include model name
        self.weight_config   = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\weight.yaml")    # includes weights + version
        self.prompt_config   = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\prompt.yaml")  
    def fn1(self):
        weights = self.weight_config["weights"]
        contribution = {}
        total = 0.0
        for section_data in self.section_outputs:
            section_name    = section_data["section"]
            total_score     = section_data["total_score"]
            section_weight  = weights[section_name]["section_weight"]
            section_contrib = total_score * section_weight
            contribution[section_name] = {
                "section_total": total_score,
                "section_weight": section_weight,
                "contribution": Helper.fop(section_contrib)
            }
            total = total + section_contrib
        return {
            "final_resume_score":Helper.fop(total),
            "section_contribution":contribution,
            "globalfeedback":self.parse
        }
    def fn2(self):
        details = {}
        for section_data in self.section_outputs:
            details[section_data["section"]] = {
                'total_score':section_data['total_score'],
                'scores':section_data['scores']
            }
        # print(f"detail -> {details}")
        prompt = f"""
        You are an expert CV and Resume reviewer.

        Your task is to generate a SINGLE, GLOBAL feedback summary based on the full resume evaluation results below.

        IMPORTANT:
        - You MUST read and consider feedback from ALL resume sections.
        - Do NOT repeat section-by-section feedback.
        - Synthesize insights into an overall assessment.
        - Assume the user will NOT read individual section details.

        Focus on:
        1. Overall strengths of the resume
        2. Key weaknesses or gaps
        3. High-impact, actionable improvement advice

        Guidelines:
        - Be professional, constructive, and specific
        - Avoid generic statements
        - Do NOT assume missing information
        - Base your feedback ONLY on the evaluation data provided

        INPUT (section-level evaluation results):
        {json.dumps(details, indent=2)}

        STRICT OUTPUT RULES:
        - Return JSON ONLY
        - No markdown
        - No explanation
        - No extra text

        Output schema:
        {{
        "response": "Concise but insightful global feedback covering strengths, weaknesses, and improvement suggestions."
        }}
        """
        self.parse,_ = self._call_raw(prompt)
        print(self.parse)
        return details
    def fn3(self):
        return {
            "model_name": self.model_config['model']['generation_model'],
            "timestamp": self.timestamp,
            "weights_version": self.weight_config.get("version", "unknown"),
            "prompt_version": self.prompt_config.get("version", "unknown")
        }
    def fn0(self):
        detail_part     = self.fn2()
        conclution_part = self.fn1()
        metadata_part   = self.fn3()
        # return {
        #     "conclution":conclution_part,
        #     "section_detail":detail_part,
        #     "metadata":metadata_part
        # }
        return {
            "conclution":conclution_part,
            "section_detail":detail_part,
            "metadata":metadata_part
        }

<hr>

In [71]:
resume_json = Helper.load_json(r"C:\Users\TunKedsaro\Desktop\CVResume\src\mock\resume3.json")
targetrole = "data scientist"
output_lang = "en"
caller    = LlmCaller()
agg       = SectionScoreAggregator()

In [72]:
p1 = PromptBuilder(
    section     = "Profile",
    criteria    = ["Completeness", "ContentQuality"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt1 = p1.build()
p2 = PromptBuilder( 
    section     = "Summary", 
    criteria    = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt2 = p2.build()
op1,raw1 = caller.call(prompt1)
op2,raw2 = caller.call(prompt2)

In [73]:
s1 = agg.aggregate(op1)
s2 = agg.aggregate(op2)
x = newga(SectionScoreAggregator_output = [s1,s2])
output = x.fn0()

{'response': "The resume effectively positions the candidate for a Data Scientist role, demonstrating a clear career focus and strong foundational communication. Both the professional summary and profile are complete, concise, and grammatically sound, clearly outlining the candidate's expertise in data analysis and driving business impact. The primary area for high-impact improvement is to enhance content quality by integrating specific methodologies, tools, and, crucially, quantifiable achievements. Incorporating concrete metrics and detailing the tangible impact of past work will significantly strengthen the overall narrative and differentiate the candidate more effectively."}


In [74]:
output

{'conclution': {'final_resume_score': 6.2,
  'section_contribution': {'Profile': {'section_total': 16.0,
    'section_weight': 0.1,
    'contribution': 1.6},
   'Summary': {'section_total': 46.0,
    'section_weight': 0.1,
    'contribution': 4.6}},
  'globalfeedback': {'response': "The resume effectively positions the candidate for a Data Scientist role, demonstrating a clear career focus and strong foundational communication. Both the professional summary and profile are complete, concise, and grammatically sound, clearly outlining the candidate's expertise in data analysis and driving business impact. The primary area for high-impact improvement is to enhance content quality by integrating specific methodologies, tools, and, crucially, quantifiable achievements. Incorporating concrete metrics and detailing the tangible impact of past work will significantly strengthen the overall narrative and differentiate the candidate more effectively."}},
 'section_detail': {'Profile': {'total_s

In [69]:
print(str(json.dumps(x,indent=4, ensure_ascii=False)))

{
    "conclution": {
        "final_resume_score": 6.4,
        "section_contribution": {
            "Profile": {
                "section_total": 16.0,
                "section_weight": 0.1,
                "contribution": 1.6
            },
            "Summary": {
                "section_total": 48.0,
                "section_weight": 0.1,
                "contribution": 4.8
            }
        },
        "globalfeedback": {
            "response": "The candidate demonstrates a strong and clear professional identity as a data scientist, with a highly relevant, concise, and grammatically flawless summary that effectively highlights their experience, technical strengths, and career focus. The overall positioning is excellent. The primary area for improvement across both the Profile and Summary sections is the inclusion of specific, quantifiable achievements, metrics, and project impacts. Adding concrete numbers and results would significantly enhance the content quality and showc

{
    "conclution": {
        "final_resume_score": 6.4,
        "section_contribution": {
            "Profile": {
                "section_total": 16.0,
                "section_weight": 0.1,
                "contribution": 1.6
            },
            "Summary": {
                "section_total": 48.0,
                "section_weight": 0.1,
                "contribution": 4.8
            }
        },
        "globalfeedback": {
            "response": "The candidate demonstrates a strong and clear professional identity as a data scientist, with a highly relevant, concise, and grammatically flawless summary that effectively highlights their experience, technical strengths, and career focus. The overall positioning is excellent. The primary area for improvement across both the Profile and Summary sections is the inclusion of specific, quantifiable achievements, metrics, and project impacts. Adding concrete numbers and results would significantly enhance the content quality and showcase the tangible value delivered by the candidate."
        }
    },
    "section_detail": {
        "Profile": {
            "total_score": 16.0,
            "scores": {
                "ContentQuality": {
                    "score": 6.0,
                    "feedback": "The summary clearly states passion and background in data science, but lacks specific quantifiable achievements or project impacts."
                },
                "Completeness": {
                    "score": 10.0,
                    "feedback": "The section effectively establishes the candidate's professional identity and clear positioning as a data scientist."
                }
            }
        },
        "Summary": {
            "total_score": 48.0,
            "scores": {
                "RoleRelevance": {
                    "score": 10.0,
                    "feedback": "Clearly highlights data analysis, experiments, and driving business growth through data, directly matching a data scientist role."
                },
                "Length": {
                    "score": 10.0,
                    "feedback": "Concise and impactful, fitting perfectly within the 2-4 sentence guideline, conveying key information effectively."
                },
                "Grammar": {
                    "score": 10.0,
                    "feedback": "Flawless grammar and clear sentence structure, enhancing readability and professionalism in the summary."
                },
                "ContentQuality": {
                    "score": 8.0,
                    "feedback": "Clearly articulates focus and value proposition, hinting at impact. Lacks specific quantifiable metrics within this section."
                },
                "Completeness": {
                    "score": 10.0,
                    "feedback": "Excellently covers all key elements: experience, technical strengths, and career focus in a concise manner."
                }
            }
        }
    },
    "metadata": {
        "model_name": "gemini-2.5-flash",
        "timestamp": "2026-01-07 18:01:45.870066+07:00",
        "weights_version": "weights_v1",
        "prompt_version": "prompt_v2"
    }
}

<hr>

In [78]:
p1 = PromptBuilder(
    section     = "Profile",
    criteria    = ["Completeness", "ContentQuality"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt1 = p1.build()

p2 = PromptBuilder( 
    section     = "Summary", 
    criteria    = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt2 = p2.build()

p3 = PromptBuilder( 
    section     = "Education", 
    criteria    = ["Completeness","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt3 = p3.build()

p4 = PromptBuilder( 
    section     = "Experience", 
    criteria    = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt4 = p4.build()

p5 = PromptBuilder( 
    section     = "Activities", 
    criteria    = ["Completeness", "ContentQuality","Grammar","Length"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt5 = p5.build()

p6 = PromptBuilder( 
    section     = "Skills", 
    criteria    = ["Completeness","Length","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt6 = p6.build()

op1,raw1 = caller.call(prompt1)
op2,raw2 = caller.call(prompt2)
op3,raw3 = caller.call(prompt3)
op4,raw4 = caller.call(prompt4)
op5,raw5 = caller.call(prompt5)
op6,raw6 = caller.call(prompt6)

s1 = agg.aggregate(op1)
s2 = agg.aggregate(op2)
s3 = agg.aggregate(op3)
s4 = agg.aggregate(op4)
s5 = agg.aggregate(op5)
s6 = agg.aggregate(op6)

x = newga(SectionScoreAggregator_output = [s1,s2,s3,s4,s5,s6])

output = x.fn0()

{'response': "This resume presents a candidate with an exceptional academic foundation, boasting highly relevant dual Master's degrees and a comprehensive suite of technical skills strongly aligned with Data Scientist roles. The overall professionalism, clarity, and grammatical accuracy are significant strengths. However, the most critical area for improvement is the consistent lack of quantifiable achievements and specific impact across the 'Profile', 'Summary', and 'Experience' sections. The current 'Profile' section is generic and lacks depth, while a crucial 'Activities' section is entirely missing, which would otherwise demonstrate practical application and initiative beyond formal employment. To significantly enhance this resume, focus on: 1) Systematically quantifying every achievement and impact within your 'Profile', 'Summary', and 'Experience' sections, using metrics, percentages, or concrete results to showcase your value. 2) Revamping the 'Profile' section to be as specific

In [79]:
output

{'conclution': {'final_resume_score': 23.4,
  'section_contribution': {'Profile': {'section_total': 8.0,
    'section_weight': 0.1,
    'contribution': 0.8},
   'Summary': {'section_total': 46.0,
    'section_weight': 0.1,
    'contribution': 4.6},
   'Education': {'section_total': 20.0,
    'section_weight': 0.2,
    'contribution': 4.0},
   'Experience': {'section_total': 40.0,
    'section_weight': 0.2,
    'contribution': 8.0},
   'Activities': {'section_total': 0.0,
    'section_weight': 0.2,
    'contribution': 0.0},
   'Skills': {'section_total': 30.0,
    'section_weight': 0.2,
    'contribution': 6.0}},
  'globalfeedback': {'response': "This resume presents a candidate with an exceptional academic foundation, boasting highly relevant dual Master's degrees and a comprehensive suite of technical skills strongly aligned with Data Scientist roles. The overall professionalism, clarity, and grammatical accuracy are significant strengths. However, the most critical area for improveme

In [80]:
print(str(json.dumps(output,indent=4, ensure_ascii=False)))

{
    "conclution": {
        "final_resume_score": 23.4,
        "section_contribution": {
            "Profile": {
                "section_total": 8.0,
                "section_weight": 0.1,
                "contribution": 0.8
            },
            "Summary": {
                "section_total": 46.0,
                "section_weight": 0.1,
                "contribution": 4.6
            },
            "Education": {
                "section_total": 20.0,
                "section_weight": 0.2,
                "contribution": 4.0
            },
            "Experience": {
                "section_total": 40.0,
                "section_weight": 0.2,
                "contribution": 8.0
            },
            "Activities": {
                "section_total": 0.0,
                "section_weight": 0.2,
                "contribution": 0.0
            },
            "Skills": {
                "section_total": 30.0,
                "section_weight": 0.2,
                "contribution"

{
    "conclution": {
        "final_resume_score": 23.4,
        "section_contribution": {
            "Profile": {
                "section_total": 8.0,
                "section_weight": 0.1,
                "contribution": 0.8
            },
            "Summary": {
                "section_total": 46.0,
                "section_weight": 0.1,
                "contribution": 4.6
            },
            "Education": {
                "section_total": 20.0,
                "section_weight": 0.2,
                "contribution": 4.0
            },
            "Experience": {
                "section_total": 40.0,
                "section_weight": 0.2,
                "contribution": 8.0
            },
            "Activities": {
                "section_total": 0.0,
                "section_weight": 0.2,
                "contribution": 0.0
            },
            "Skills": {
                "section_total": 30.0,
                "section_weight": 0.2,
                "contribution": 6.0
            }
        },
        "globalfeedback": {
            "response": "This resume presents a candidate with an exceptional academic foundation, boasting highly relevant dual Master's degrees and a comprehensive suite of technical skills strongly aligned with Data Scientist roles. The overall professionalism, clarity, and grammatical accuracy are significant strengths. However, the most critical area for improvement is the consistent lack of quantifiable achievements and specific impact across the 'Profile', 'Summary', and 'Experience' sections. The current 'Profile' section is generic and lacks depth, while a crucial 'Activities' section is entirely missing, which would otherwise demonstrate practical application and initiative beyond formal employment. To significantly enhance this resume, focus on: 1) Systematically quantifying every achievement and impact within your 'Profile', 'Summary', and 'Experience' sections, using metrics, percentages, or concrete results to showcase your value. 2) Revamping the 'Profile' section to be as specific and impactful as your professional 'Summary', integrating explicit achievements and their measured outcomes. 3) Creating an 'Activities' section to highlight relevant projects, hackathons, open-source contributions, or competitions, providing tangible evidence of your applied skills and passion for data science."
        }
    },
    "section_detail": {
        "Profile": {
            "total_score": 8.0,
            "scores": {
                "ContentQuality": {
                    "score": 2.0,
                    "feedback": "Generic statements lack specific achievements, methods, tools, or quantified impact, failing to demonstrate concrete expertise."
                },
                "Completeness": {
                    "score": 6.0,
                    "feedback": "Contains all expected elements like identity and career focus, but the detail is superficial, limiting full understanding."
                }
            }
        },
        "Summary": {
            "total_score": 46.0,
            "scores": {
                "RoleRelevance": {
                    "score": 10.0,
                    "feedback": "Clearly defines the candidate's expertise in data analysis, experimentation, and data science, directly aligning with the target role and business impact."
                },
                "Length": {
                    "score": 10.0,
                    "feedback": "The two-sentence summary is concise and provides relevant information without being overly verbose or too brief."
                },
                "Grammar": {
                    "score": 10.0,
                    "feedback": "The summary is well-written with no grammatical errors, ensuring clarity and professionalism."
                },
                "ContentQuality": {
                    "score": 6.0,
                    "feedback": "While clear on methods and general impact, the summary lacks specific quantifiable results or explicit mention of tools within its brief structure."
                },
                "Completeness": {
                    "score": 10.0,
                    "feedback": "The summary effectively presents the candidate's core strengths, career focus, and relevant experience, meeting all expected elements for this section."
                }
            }
        },
        "Education": {
            "total_score": 20.0,
            "scores": {
                "RoleRelevance": {
                    "score": 10.0,
                    "feedback": "Excellent educational background with two highly relevant Master's degrees demonstrating strong foundational and specialized knowledge in data science and machine learning."
                },
                "Completeness": {
                    "score": 10.0,
                    "feedback": "The section is comprehensive, providing detailed information on institutions, degrees, dates, GPA, and relevant coursework for each academic pursuit."
                }
            }
        },
        "Experience": {
            "total_score": 40.0,
            "scores": {
                "RoleRelevance": {
                    "score": 8.0,
                    "feedback": "The candidate's specific data scientist roles are highly relevant, detailing key tasks and tools, though short-term."
                },
                "Length": {
                    "score": 8.0,
                    "feedback": "Each entry is concisely presented with relevant information, avoiding redundancy or being overly short."
                },
                "Grammar": {
                    "score": 10.0,
                    "feedback": "Excellent grammar, spelling, and sentence structure throughout the section, ensuring high readability."
                },
                "ContentQuality": {
                    "score": 6.0,
                    "feedback": "Good action, method, and tool descriptions are present, but specific quantifiable results for data science impact are limited."
                },
                "Completeness": {
                    "score": 8.0,
                    "feedback": "All key elements like job details, clear bullet points, and relevant tools are present, offering a good overview."
                }
            }
        },
        "Activities": {
            "total_score": 0.0,
            "scores": {
                "Length": {
                    "score": 0.0,
                    "feedback": "The 'Activities' section is entirely missing from the provided resume."
                },
                "Grammar": {
                    "score": 0.0,
                    "feedback": "The 'Activities' section is entirely missing from the provided resume."
                },
                "ContentQuality": {
                    "score": 0.0,
                    "feedback": "The 'Activities' section is entirely missing from the provided resume."
                },
                "Completeness": {
                    "score": 0.0,
                    "feedback": "The 'Activities' section is entirely missing from the provided resume. This section would typically highlight competitions, hackathons, or club involvement."
                }
            }
        },
        "Skills": {
            "total_score": 30.0,
            "scores": {
                "RoleRelevance": {
                    "score": 10.0,
                    "feedback": "Comprehensive technical skills (ML, Big Data, Cloud) and relevant tools are strongly aligned with a Data Scientist role."
                },
                "Length": {
                    "score": 10.0,
                    "feedback": "The section is appropriately detailed, providing extensive, relevant information without being overly long or redundant."
                },
                "Completeness": {
                    "score": 10.0,
                    "feedback": "All key elements are present, including technical skills, relevant tools, soft skills, and programming language proficiency."
                }
            }
        }
    },
    "metadata": {
        "model_name": "gemini-2.5-flash",
        "timestamp": "2026-01-07 18:18:20.405035+07:00",
        "weights_version": "weights_v1",
        "prompt_version": "prompt_v2"
    }
}
